In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join("..")))
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from src.data_loader import load_sales, load_purchases, load_products
from src.demand_forecast import calculate_forecast
from src.stock_manager import calculate_min_stock
from src.purchase_suggester import generate_purchase_suggestion
from src.supplier_selector import select_suppliers


In [3]:
# === Opciones disponibles ===
coverage_days_options = [7, 15, 21, 30]
forecast_methods = ["mean", "rolling", "weighted", "trend"]

# === Widgets ===
coverage_dropdown = widgets.Dropdown(
    options=coverage_days_options,
    value=7,
    description="Cobertura (días):",
    style={'description_width': 'initial'}
)

method_dropdown = widgets.Dropdown(
    options=forecast_methods,
    value="mean",
    description="Método forecast:",
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description="Generar Sugerido",
    button_style="success"
)

output = widgets.Output()

# === Función a ejecutar ===
def run_forecast(b):
    with output:
        output.clear_output()
        try:
            coverage_days = coverage_dropdown.value
            method = method_dropdown.value

            print(f"⚡ Ejecutando forecast con {coverage_days} días y método '{method}'...\n")

            # === Cargar datos ===
            sales = load_sales("../data/sales.csv")
            products = load_products("../data/products.csv")      # incluye stock y datos producto
            purchases = load_purchases("../data/compras.csv")   # historial de compras

            # === Forecast (ventas diarias) ===
            forecast = calculate_forecast(
                sales,
                days=60,   # usa últimos 60 días para suavizar outliers
                method=method,
                window=7
            )

            # === Generar sugerido ===
            suggestion = generate_purchase_suggestion(
                sales_df=sales,
                forecast_df=forecast,
                stock_df=products,      # productos contiene current_stock
                products_df=products,   # productos también tiene code y name
                purchases_df=purchases,
                coverage_days=coverage_days,
                safety_days=3           # stock mínimo para 2–3 días de seguridad
            )

            if suggestion.empty:
                print("⚠️ No hay datos para mostrar (revisa tus CSV o filtros).")
            else:
                display(suggestion.head(20))

        except Exception as e:
            print("❌ Error en la ejecución:", e)

# Conectar botón a función
run_button.on_click(run_forecast)



# Mostrar widgets
display(coverage_dropdown, method_dropdown, run_button, output)

Dropdown(description='Cobertura (días):', options=(7, 15, 21, 30), style=DescriptionStyle(description_width='i…

Dropdown(description='Método forecast:', options=('mean', 'rolling', 'weighted', 'trend'), style=DescriptionSt…

Button(button_style='success', description='Generar Sugerido', style=ButtonStyle())

Output()

,product_code,product_name,current_stock,min_stock,expected_demand,suggested_purchase,best_supplier,best_cost,est_total_cost
0,000002,MARG. BONELLA VITAL SCH 50G 67242662,5.0,5,9,8,DISPROVIEC,0.27,2.17
1,000082,HAR. VIDASOYA ORIENTAL 200GR 903002001,2.0,4,7,8,TAITAB S.A.S.,0.78,6.24
2,000087,MAICENA ORIENTAL 200GR 904001006,4.0,4,10,10,TAITAB S.A.S.,0.78,7.80
3,000076,SALSA CHINA ORIENTAL 200ML 902002011,4.0,5,14,14,TAITAB S.A.S.,1.25,17.50
4,000077,SALSA CHINA ORIENTAL 100ML 902002013,9.0,6,15,12,VERA LOZA SAMUEL ALEJANDRO,0.73,8.71
5,000934,FID. RAPIDITO ORIENTAL POLLO 100G,0.0,7,18,24,DEVIES CORP S.A.,0.57,13.56
6,000079,SALSA DE AJI ORIENTAL 100ML 902001003,13.0,5,11,3,VERA LOZA SAMUEL ALEJANDRO,0.75,2.26
7,000042,DEO. REXONA R/O V8 50ML 84152921,0.0,4,10,13,TAITAB S.A.S.,1.90,24.70
8,000394,PAP HIG FAMILIA ACOL MEGARROLLO 4X12 ROLL TIRA,1.0,3,7,9,UNKNOWN,,NaN
9,000390,PAP HIG FAMILIA TRIPLE PLUS 2EN1 4PQX12TIRA,141.0,8,19,0,DIMEVAR CIA LTDA,0.29,0.00


In [2]:
product = load_products('../data/products.csv')
purchases = load_purchases('../data/compras.csv') 

suppliers = select_suppliers(purchases)
pid = '0000004598'
print(suppliers[suppliers["product_id"].str.contains(pid, na=False)])

print("👉 Compras originales:")
display(purchases[purchases["product_id"] == pid][["purchase_date","supplier_name","quantity_purchased","total_amount"]])

print("\n👉 Resultado en suppliers:")
display(suppliers[suppliers["product_id"] == pid])

     product_id best_supplier  best_cost
579  0000004598      COILE SA      0.355
👉 Compras originales:


,purchase_date,supplier_name,quantity_purchased,total_amount
41,2025-02-06,COILE SA,24.0,8.33
42,2025-03-13,COILE SA,24.0,8.50
43,2025-05-08,COILE SA,24.0,8.50
44,2025-06-05,COILE SA,12.0,4.26



👉 Resultado en suppliers:


,product_id,best_supplier,best_cost
579,0000004598,COILE SA,0.355
